In [2]:
from geopy.geocoders import Nominatim
import pandas as pd
import time

# Load dataset
data = pd.read_csv("data.csv")

# Initialize geocoder
geolocator = Nominatim(user_agent="destination-recommender")

# Function to fetch latitude and longitude
def fetch_coordinates(location_name):
    try:
        location = geolocator.geocode(location_name)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
        time.sleep(1)
    except Exception as e:
        print(f"Error fetching coordinates for {location_name}: {e}")
        return None, None


failed_locations = []

def fetch_coordinates_safe(location_name):
    try:
        lat, lon = fetch_coordinates(location_name)
        if lat is None or lon is None:
            failed_locations.append(location_name)
        return lat, lon
    except Exception as e:
        failed_locations.append(location_name)
        return None, None

# Example: Save to a local cache file
cache = {}

def fetch_with_cache(location_name):
    if location_name in cache:
        return cache[location_name]
    lat, lon = fetch_coordinates(location_name)
    cache[location_name] = (lat, lon)
    return lat, lon

# Save cache to disk
import pickle
with open("geocode_cache.pkl", "wb") as f:
    pickle.dump(cache, f)

# Add latitude and longitude columns
data[['latitude', 'longitude']] = data['pName'].apply(lambda x: pd.Series(fetch_coordinates(x)))

# Save the updated dataset
data.to_csv("destinations_with_coordinates.csv", index=False)

print("Coordinates added to the dataset!")


Coordinates added to the dataset!
